# Adding Array Parameters

We start by creating a simple simulation and adding REBOUNDx

In [1]:
import rebound
import reboundx
import numpy as np

sim = rebound.Simulation()
sim.add(m=1., hash="star")
sim.add(a=1., hash="planet")
ps = sim.particles
p = ps["planet"]
rebx = reboundx.Extras(sim)

We can add array parameters to particles and effects by simply assigning them numpy arrays:

In [2]:
doubles = np.array([1.,2.,3.])
p.params["a"] = doubles

and we access them as usual:

In [3]:
p.params["a"]

array([ 1.,  2.,  3.])

In order to maintain a consistent syntax, lists won't work:

In [4]:
try:
    p.params["list"] = [1.,2.,3.]
except AttributeError as e:
    print(e)

Can't assign lists as params. Use numpy arrays instead. See documentation.


# Note on memory

REBOUNDx allocates and manages the memory for all parameters, copying the values from any python objects (like the array `doubles`). This means that any changes in `doubles` will not be reflected in the parameter:

In [5]:
doubles[0] = 5.
p.params["a"]

array([ 1.,  2.,  3.])

We can always get a reference and update that:

In [6]:
doubles = p.params["a"]
doubles[0] = 5.
p.params["a"]

array([ 5.,  2.,  3.])

# Arrays of Objects

If your array holds anything other than basic python types, you need to make sure that you pass `dtype=object`. One possible pitfall of this type are REBOUNDx hashes, which are `ctypes` objects. To store the hashes, we would do:

In [7]:
p.params["hashes"] = np.array([sim.particles[0].hash, sim.particles[1].hash], dtype=object)

For example, we can check that accessing the star through the corresponding hash in this array gives us a particle with the right mass (1.):

In [8]:
starhash = p.params["hashes"][0]
print("Particle's mass = {0}".format(sim.particles[starhash].m))

Particle's mass = 1.0


We could similarly add a parameter holding a snapshot of all the current particles' orbits. Since we just have one planet in this simulation, this is just an array of length 1:

In [9]:
p.params["orbitarray"] = np.array(sim.calculate_orbits(), dtype=object)
print("The planet's semimajor axis is {0}".format(p.params["orbitarray"][0].a))

The planet's semimajor axis is 1.0


As always, you also need to make sure the code supports that object type--REBOUNDx will raise an exception if the object type is not supported:

In [10]:
try:
    p.params["error"] = np.array([sim], dtype=object)
except AttributeError as e:
    print(e)

REBOUNDx Error: Data type <class 'rebound.simulation.Simulation'> for param 'error' not supported.


# Multidimensional arrays

Multidimensional arrays also work:

In [11]:
p.params["multi"] = np.array([[[1,2,3,4],[10,20,30,40]], [[5,6,7,8],[50,60,70,80]], [[9,10,11,12], [90,100,110,120]]])
p.params["multi"]

array([[[  1,   2,   3,   4],
        [ 10,  20,  30,  40]],

       [[  5,   6,   7,   8],
        [ 50,  60,  70,  80]],

       [[  9,  10,  11,  12],
        [ 90, 100, 110, 120]]])

In [12]:
p.params["multi"].shape

(3, 2, 4)